# `SELECT`

Nesta aula vamos explorar o comando `SELECT`, usado para consultar a base de dados. 

## Explorando a estrutura da base de dados

vamos a alguns comandos básicos para explorar e entender melhor uma base de dados na primeira consulta

In [28]:
# Configuração inicial do banco de dados
import sqlite3
import os
import sys

# Adicionando o diretório de testes ao path
from tests.tests_select import *

# Conectando ao banco de dados
conn = sqlite3.connect('consultoria.db')

# Habilitando suporte a chaves estrangeiras (muito importante!)
conn.execute("PRAGMA foreign_keys = ON;")

print("✅ Conectado ao banco de dados 'consultoria.db'")
print("✅ Chaves estrangeiras habilitadas")


def validate_and_execute(query, test_function):
    """Valida query em ambiente isolado antes de executar no banco principal"""
    if not query or not query.strip():
        print("❌ Query vazia! Escreva sua query SQL antes de executar.")
        return False
    
    
    try:
        # Testar a query em um banco temporário isolado
        test_conn = sqlite3.connect(':memory:')
        test_conn.execute("PRAGMA foreign_keys = ON;")
        
        # Copiar estrutura existente para o banco de teste
        for linha in conn.iterdump():
            if not linha.startswith('BEGIN') and not linha.startswith('COMMIT'):
                try:
                    test_conn.execute(linha)
                except:
                    pass  # Ignora erros de estrutura já existente
        
        # Testar a query do usuário no banco isolado
        test_conn.execute(query)
        test_conn.commit()
        
        # Se chegou até aqui, a query é válida sintaticamente
        # Agora executar no banco principal
        conn.execute(query)
        conn.commit()
        
        # Testar se o resultado está correto
        success = test_function(conn,query)
        
        if success:
            print("✅ Query executada e validada com sucesso!")
            return True
        else:
            # Se teste falhar, fazer rollback
            print("💡 Query executada mas resultado incorreto. Banco restaurado, tente novamente!")
            return False
            
    except Exception as e:
        print(f"❌ Erro na query: {str(e)}")
        print("💡 Corrija a sintaxe e tente novamente!")
        return False
    finally:
        if 'test_conn' in locals():
            test_conn.close()


def execute_query(query):
    """Executa uma query e exibe os resultados de forma formatada"""
    if not query or not query.strip():
        print("❌ Query vazia! Forneça uma query SQL para executar.")
        return
    
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        
        # Determinar se é um SELECT ou outro tipo de query
        query_type = query.strip().upper().split()[0]
        
        if query_type == 'SELECT':
            # Para SELECT, buscar e exibir os resultados
            results = cursor.fetchall()
            
            # Obter nomes das colunas
            column_names = [description[0] for description in cursor.description]
            
            if not results:
                print("📄 Nenhum resultado encontrado.")
                return
            
            # Calcular largura das colunas para formatação
            col_widths = []
            for i, col_name in enumerate(column_names):
                max_width = len(col_name)
                for row in results:
                    if row[i] is not None:
                        max_width = max(max_width, len(str(row[i])))
                col_widths.append(max_width + 2)  # +2 para espaçamento
            
            # Imprimir cabeçalho
            print("\n" + "=" * (sum(col_widths) + len(column_names) - 1))
            header = "|".join(col_name.ljust(col_widths[i]) for i, col_name in enumerate(column_names))
            print(header)
            print("-" * (sum(col_widths) + len(column_names) - 1))
            
            # Imprimir dados
            for row in results:
                formatted_row = "|".join(
                    str(value).ljust(col_widths[i]) if value is not None else "NULL".ljust(col_widths[i])
                    for i, value in enumerate(row)
                )
                print(formatted_row)
            
            print("=" * (sum(col_widths) + len(column_names) - 1))
            print(f"📊 Total de registros: {len(results)}")
            
        else:
            # Para INSERT, UPDATE, DELETE, etc.
            conn.commit()
            rows_affected = cursor.rowcount
            
            if query_type in ['INSERT', 'UPDATE', 'DELETE']:
                print(f"✅ {query_type} executado com sucesso!")
                if rows_affected >= 0:
                    print(f"📈 Linhas afetadas: {rows_affected}")
            else:
                print(f"✅ Comando {query_type} executado com sucesso!")
                
    except Exception as e:
        print(f"❌ Erro ao executar query: {str(e)}")
    finally:
        if 'cursor' in locals():
            cursor.close()


def test_and_rollback(test_function):
    """Mantido para compatibilidade - executa teste simples"""
    success = test_function(conn)
    if not success:
        print("💡 Execute a query correta primeiro!")
    return success

print("Sistema configurado")

✅ Conectado ao banco de dados 'consultoria.db'
✅ Chaves estrangeiras habilitadas
Sistema configurado


### O que é um Schema?

O schema é a estrutura que define como os dados são organizados em um banco de dados. É como a "planta baixa" que mostra:

1. **Tabelas**: Quais "gavetas" temos para guardar os dados
   - Ex: `clientes`, `projetos`, `consultores`

2. **Colunas**: Que informações guardamos em cada tabela
   - Ex: em `clientes` temos: id, nome, setor, cidade

3. **Tipos de Dados**: Que tipo de valor cada coluna aceita
   - `INTEGER`: números inteiros (1, 2, 3)
   - `TEXT`: textos ("Ana", "São Paulo")
   - `DATE`: datas (2024-01-15)

4. **Relacionamentos**: Como as tabelas se conectam
   - Ex: cada `projeto` pertence a um `cliente`

### Comandos para Explorar o Schema

Em outros bancos SQL (como MySQL), usamos:
- `SHOW TABLES;` para listar todas as tabelas
- `DESCRIBE tabela;` para ver a estrutura de uma tabela

Porém, no SQLite esses comandos não existem! 😅 
Mas conseguimos o mesmo resultado usando:

```sql
-- Equivalente ao SHOW TABLES:
SELECT name FROM sqlite_master WHERE type='table';

-- Equivalente ao DESCRIBE tabela:
PRAGMA table_info(nome_da_tabela);
```

### 🎯 Antes de Começar com SELECT

Antes de fazer nossas primeiras consultas com `SELECT`, é importante entender bem a estrutura do banco. 
Vamos usar os comandos acima para explorar:
1. Quais tabelas temos disponíveis
2. Que campos cada tabela possui
3. Como as tabelas se relacionam

Isso vai nos ajudar a fazer consultas mais eficientes!

### 🔍 Estrutura das Tabelas (equivalente ao DESCRIBE)

# Diagrama do Banco de Dados

![Diagrama EER](assets/diagram.png)



In [29]:
execute_query("SELECT * FROM clientes")

❌ Erro ao executar query: database is locked



nome                 
---------------------
TechCorp Solutions   
Verde Agro Ltda      
MetalMax Indústrias  
EduCare Ensino       
FastLogistic S.A.    
📊 Total de registros: 5


In [30]:
#Preencha a query:
select_x_query = '''
    SELECT setor,cidade,uf FROM clientes
'''

validate_and_execute(select_x_query, test_select_scu_clientes)

❌ Erro na query: database is locked
💡 Corrija a sintaxe e tente novamente!


False

In [26]:
execute_query("""
INSERT INTO clientes(nome, setor, cidade, uf)
    VALUES
    ('Mineração Ouro Verde', 'Mineração', 'Belo Horizonte', 'MG'),
    ('Café Premium Montanhas', 'Agronegócio', 'Uberlândia', 'MG'),
    ('PetroSul Distribuidora', 'Petróleo', 'Rio de Janeiro', 'RJ'),
    ('BeachTech Inovação', 'Tecnologia', 'Niterói', 'RJ'),
    ('TurisRio Hospedagem', 'Turismo', 'Cabo Frio', 'RJ')
""")

❌ Erro ao executar query: database is locked
